# 模組 1: 文檔攝取與處理基礎
## Module 1: Document Ingestion and Processing Fundamentals

> **教學目標**: 掌握企業級文檔攝取流程、了解 Docling 深度整合、實作語義分塊演算法

---

## 📚 學習成果 (Learning Outcomes)

完成本模組後，您將能夠：
1. **設計文檔攝取架構** - 建立可擴展的文檔處理管線
2. **應用 Docling 進行文檔解析** - 掌握 PDF/DOCX 精確提取
3. **實作語義分塊演算法** - 基於 embeddings 的智能分塊
4. **建立品質檢測機制** - 自動化的文檔品質評估

---

## 🎯 核心概念 (Core Concepts)

### 1. 企業級文檔攝取挑戰
- 多格式支援 (PDF, DOCX, PPTX, HTML)
- 大規模並發處理
- 錯誤處理與恢復機制
- 品質保證與驗證

### 2. 運算思維應用
- **分解**: 文檔處理流程模組化
- **模式識別**: 文檔結構模式
- **抽象化**: 統一文檔處理介面
- **演算法**: 自適應分塊策略

## 🔧 環境設定與依賴安裝

In [36]:
# # 修正依賴版本兼容性問題
# !pip install "numpy<2" --upgrade

# # 安裝核心依賴套件 (指定兼容版本)
# !pip install docling python-docx PyPDF2 sentence-transformers textstat
# !pip install plotly pandas "numpy<2" scikit-learn
# !pip install spacy language-tool-python

# # 如果 spaCy 模型未安裝，則安裝
# !python -m spacy download en_core_web_sm || echo "spaCy 模型下載失敗，將使用降級處理方法"

# print("✅ 依賴套件安裝完成（修正版本兼容性）")

In [37]:
# 導入必要的函式庫
import os
import sys
import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# 基礎文檔處理 (先不導入 Docling，避免相容性問題)
try:
    from docling.document_converter import DocumentConverter
    DOCLING_AVAILABLE = True
    print("✅ Docling 可用")
except Exception as e:
    print(f"⚠️  Docling 導入失敗: {e}")
    print("📝 將使用替代的文檔處理方法")
    DOCLING_AVAILABLE = False
    
# 替代文檔處理
import PyPDF2
from docx import Document as DocxDocument

# NLP 與語義處理
try:
    from sentence_transformers import SentenceTransformer
    EMBEDDINGS_AVAILABLE = True
    print("✅ SentenceTransformers 可用")
except Exception as e:
    print(f"⚠️  SentenceTransformers 導入失敗: {e}")
    EMBEDDINGS_AVAILABLE = False

try:
    import spacy
    # 嘗試載入模型
    try:
        nlp = spacy.load('en_core_web_sm')
        SPACY_AVAILABLE = True
        print("✅ spaCy 模型可用")
    except OSError:
        print("⚠️  spaCy 英文模型未安裝，請執行: python -m spacy download en_core_web_sm")
        SPACY_AVAILABLE = False
except ImportError:
    SPACY_AVAILABLE = False

from sklearn.metrics.pairwise import cosine_similarity
import textstat

# 可視化
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 設定專案路徑
PROJECT_ROOT = Path('/home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance')
sys.path.append(str(PROJECT_ROOT))

print("✅ 環境設定完成")
print(f"📂 專案根目錄: {PROJECT_ROOT}")
print(f"🔧 可用組件: Docling={DOCLING_AVAILABLE}, Embeddings={EMBEDDINGS_AVAILABLE}, spaCy={SPACY_AVAILABLE}")

✅ Docling 可用
✅ SentenceTransformers 可用
✅ spaCy 模型可用
✅ 環境設定完成
📂 專案根目錄: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance
🔧 可用組件: Docling=True, Embeddings=True, spaCy=True


In [38]:
# !python3 -m spacy download en_core_web_sm

## 📋 資料結構設計

### 標準化文檔元資料模型

In [39]:
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict
from datetime import datetime

# ==========================================================
# 一、DocumentMetadata：文檔標準化元資料結構
# ==========================================================
@dataclass
class DocumentMetadata:
    """
    📘 文檔的標準化元資料結構
    --------------------------------------------------
    用於描述一份文檔的整體屬性，例如：
    - 檔名、作者、建立時間、大小、語言、分類等。
    - 通常作為文件管理系統或知識庫的上層描述。
    """

    document_id: str               # 文檔唯一識別碼（UUID或系統編號）
    title: str                     # 文檔標題
    authors: List[str]             # 作者清單
    created_date: Optional[datetime]  # 建立日期
    modified_date: Optional[datetime] # 最後修改日期
    document_type: str             # 文件類型（如 'paper', 'report', 'policy'）
    file_path: str                 # 檔案儲存路徑
    file_size: int                 # 檔案大小（單位：bytes）
    page_count: int                # 頁數
    language: str                  # 語言（如 'zh-TW', 'en'）
    keywords: List[str]            # 關鍵字列表
    category: str                  # 文檔分類
    quality_score: Optional[float] = None  # 文件品質分數（0～1）
    processing_status: str = 'pending'     # 處理狀態：'pending' | 'processing' | 'completed' | 'failed'

    def to_dict(self) -> Dict:
        """轉換為字典格式，用於序列化或API輸出"""
        return asdict(self)


# ==========================================================
# 二、ProcessedChunk：文檔分塊結構
# ==========================================================
@dataclass
class ProcessedChunk:
    """
    🧱 文檔分塊後的語義單元
    --------------------------------------------------
    文檔分割（chunking）後的最小語義片段。
    每個分塊可進行向量嵌入，用於 RAG 或檢索任務。
    """

    chunk_id: str                  # 分塊唯一識別碼
    document_id: str               # 所屬文檔ID（對應 DocumentMetadata.document_id）
    content: str                   # 該分塊的文字內容
    chunk_index: int               # 在整份文檔的順序（0起算）
    start_page: int                # 起始頁碼
    end_page: int                  # 結束頁碼
    word_count: int                # 該分塊的字數
    embedding: Optional[List[float]] = None  # 該分塊的向量嵌入（語義表示）
    semantic_type: str = 'text'    # 分塊類型：'text' | 'table' | 'figure' | 'header'


# ==========================================================
# 三、QualityMetrics：文檔品質指標
# ==========================================================
@dataclass  
class QualityMetrics:
    """
    📊 文檔品質評估指標
    --------------------------------------------------
    用於量化文檔或分塊的內容品質：
    - 幫助自動評估OCR正確率、結構完整性與可讀性。
    """

    accuracy_score: float          # 文本準確性（拼字或辨識正確率）[0~1]
    completeness_score: float      # 完整性（是否有缺漏）[0~1]
    readability_score: float       # 可讀性（語法與表達流暢度）[0~1]
    structure_score: float         # 結構完整性（標題/段落/表格）[0~1]
    overall_score: float           # 綜合品質分數（通常為加權平均）[0~1]


# ==========================================================
# 範例使用情境（文件 → 分塊 → 品質評估）
# ==========================================================
if __name__ == "__main__":
    # 建立文件元資料
    doc_meta = DocumentMetadata(
        document_id="DOC-001",
        title="AI 報告範例",
        authors=["Sunny", "Alice"],
        created_date=datetime(2025, 11, 11, 10, 0),
        modified_date=datetime(2025, 11, 11, 12, 0),
        document_type="report",
        file_path="/data/docs/ai_report.pdf",
        file_size=204800,
        page_count=12,
        language="zh-TW",
        keywords=["AI", "NLP", "RAG"],
        category="技術文件",
        quality_score=None,
        processing_status="pending"
    )

    # 模擬一個分塊
    chunk = ProcessedChunk(
        chunk_id="CHUNK-001",
        document_id="DOC-001",
        content="生成式 AI 是 2020 年代最具影響力的技術之一...",
        chunk_index=0,
        start_page=1,
        end_page=1,
        word_count=15,
        embedding=[0.12, 0.56, 0.88],  # 簡化範例
        semantic_type="text"
    )

    # 模擬品質評估
    metrics = QualityMetrics(
        accuracy_score=0.95,
        completeness_score=0.9,
        readability_score=0.85,
        structure_score=0.88,
        overall_score=0.89
    )

    print("✅ 文檔元資料：", doc_meta.to_dict())
    print("🧩 文檔分塊內容：", chunk)
    print("📊 品質評估指標：", metrics)


✅ 文檔元資料： {'document_id': 'DOC-001', 'title': 'AI 報告範例', 'authors': ['Sunny', 'Alice'], 'created_date': datetime.datetime(2025, 11, 11, 10, 0), 'modified_date': datetime.datetime(2025, 11, 11, 12, 0), 'document_type': 'report', 'file_path': '/data/docs/ai_report.pdf', 'file_size': 204800, 'page_count': 12, 'language': 'zh-TW', 'keywords': ['AI', 'NLP', 'RAG'], 'category': '技術文件', 'quality_score': None, 'processing_status': 'pending'}
🧩 文檔分塊內容： ProcessedChunk(chunk_id='CHUNK-001', document_id='DOC-001', content='生成式 AI 是 2020 年代最具影響力的技術之一...', chunk_index=0, start_page=1, end_page=1, word_count=15, embedding=[0.12, 0.56, 0.88], semantic_type='text')
📊 品質評估指標： QualityMetrics(accuracy_score=0.95, completeness_score=0.9, readability_score=0.85, structure_score=0.88, overall_score=0.89)


## 🔧 核心文檔處理器實作

### 企業級文檔處理器

In [40]:
class EnterpriseDocumentProcessor:
    """
    企業級文檔處理器
    整合 Docling、語義分塊、品質評估
    """
    
    def __init__(self, config: Dict = None):
        """初始化處理器"""
        self.config = config or self._default_config()
        
        # 初始化 Docling 轉換器（如果可用）
        if DOCLING_AVAILABLE:
            try:
                self.doc_converter = DocumentConverter()
                print("✅ Docling 轉換器初始化完成")
            except Exception as e:
                print(f"⚠️  Docling 初始化失敗: {e}")
                self.doc_converter = None
        else:
            self.doc_converter = None
        
        # 載入語義模型（優化CUDA設定）
        if EMBEDDINGS_AVAILABLE:
            try:
                print("🔄 載入語義模型 (優化CUDA設定)...")
                
                # CUDA記憶體優化設定
                import torch
                if torch.cuda.is_available():
                    # 清理GPU記憶體
                    torch.cuda.empty_cache()
                    
                    # 檢查可用GPU
                    gpu_count = torch.cuda.device_count()
                    print(f"發現 {gpu_count} 張GPU")
                    
                    # 選擇記憶體使用較少的GPU
                    best_gpu = 1 if gpu_count > 1 else 0  # 優先使用GPU 1
                    device_name = f"cuda:{best_gpu}"
                    
                    # 設定記憶體分配策略
                    torch.cuda.set_per_process_memory_fraction(0.7, device=best_gpu)
                    print(f"使用設備: {device_name}")
                else:
                    device_name = "cpu"
                    print("CUDA不可用，使用CPU")
                
                self.semantic_model = SentenceTransformer(
                    'sentence-transformers/all-MiniLM-L6-v2',
                    device=device_name,
                    trust_remote_code=False
                )
                print(f"✅ 語義模型載入完成，設備: {self.semantic_model.device}")
                
            except Exception as e:
                print(f"⚠️  語義模型載入失敗，使用CPU: {e}")
                try:
                    # 降級到CPU
                    self.semantic_model = SentenceTransformer(
                        'sentence-transformers/all-MiniLM-L6-v2',
                        device='cpu',
                        trust_remote_code=False
                    )
                    print("✅ 語義模型載入完成 (CPU模式)")
                except Exception as e2:
                    print(f"❌ CPU模式也失敗: {e2}")
                    self.semantic_model = None
        else:
            self.semantic_model = None
        
        # 載入 spaCy 模型（如果可用）
        if SPACY_AVAILABLE:
            self.nlp = nlp
            print("✅ spaCy 模型載入完成")
        else:
            self.nlp = None
        
        # 初始化品質閾值
        self.quality_thresholds = {
            'min_overall_score': 0.7,
            'min_readability': 0.6,
            'min_completeness': 0.8
        }
        
        print(f"✅ DocumentProcessor 初始化完成")
    
    def _default_config(self) -> Dict:
        """默認配置"""
        return {
            'chunk_size': 512,
            'chunk_overlap': 50,
            'similarity_threshold': 0.3,
            'min_chunk_words': 20,
            'max_chunk_words': 1000
        }
    
    def extract_metadata(self, file_path: str) -> DocumentMetadata:
        """
        提取文檔基本元資料
        """
        file_path = Path(file_path)
        
        # 基本檔案資訊
        file_stats = file_path.stat()
        
        # 生成文檔 ID
        document_id = f"doc_{file_path.stem}_{int(datetime.now().timestamp())}"
        
        # 嘗試解析文檔獲取詳細資訊
        try:
            title, page_count = self._extract_basic_info(file_path)
        except Exception as e:
            print(f"⚠️  基本資訊提取失敗: {e}")
            title = file_path.stem
            page_count = 1
        
        return DocumentMetadata(
            document_id=document_id,
            title=title,
            authors=[],  # 需要進一步 NER 提取
            created_date=datetime.fromtimestamp(file_stats.st_ctime),
            modified_date=datetime.fromtimestamp(file_stats.st_mtime),
            document_type=self._classify_document_type(file_path),
            file_path=str(file_path),
            file_size=file_stats.st_size,
            page_count=page_count,
            language='en',  # 可擴展為自動檢測
            keywords=[],  # 需要進一步 NLP 提取
            category=self._extract_category_from_path(file_path),
            processing_status='pending'
        )
    
    def _extract_basic_info(self, file_path: Path) -> Tuple[str, int]:
        """提取基本檔案資訊"""
        title = file_path.stem
        page_count = 1
        
        # 如果是文本檔案，直接讀取
        if file_path.suffix.lower() == '.txt':
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                # 嘗試提取第一行作為標題
                lines = content.split('\n')
                first_line = lines[0].strip() if lines else ""
                if first_line and len(first_line) < 100:
                    title = first_line
                # 估算頁數（每頁約500字）
                word_count = len(content.split())
                page_count = max(1, word_count // 500)
        
        elif file_path.suffix.lower() == '.pdf' and self.doc_converter:
            # 使用 Docling 處理 PDF
            try:
                converted_doc = self.doc_converter.convert(str(file_path)).document
                page_count = len(converted_doc.pages) if hasattr(converted_doc, 'pages') else 1
                raw_text = converted_doc.export_to_text()
                first_line = raw_text.split('\n')[0].strip()
                if first_line and len(first_line) < 200:
                    title = first_line
            except Exception as e:
                print(f"⚠️  Docling PDF 處理失敗: {e}")
        
        return title, page_count
    
    def _classify_document_type(self, file_path: Path) -> str:
        """根據路徑和檔名分類文檔類型"""
        path_str = str(file_path).lower()
        
        if 'paper' in path_str or 'arxiv' in path_str:
            return 'paper'
        elif 'report' in path_str:
            return 'report'
        elif 'policy' in path_str or 'guideline' in path_str:
            return 'policy'
        else:
            return 'document'
    
    def _extract_category_from_path(self, file_path: Path) -> str:
        """從檔案路徑提取分類"""
        path_parts = file_path.parts
        
        # 尋找類似分類的目錄名稱
        for part in path_parts:
            if any(keyword in part.lower() for keyword in 
                   ['model', 'infrastructure', 'language', 'multimodal']):
                return part
        
        return 'general'
    
    def process_document(self, file_path: str) -> Tuple[DocumentMetadata, List[ProcessedChunk], QualityMetrics]:
        """
        完整的文檔處理流程
        
        Returns:
            Tuple[DocumentMetadata, List[ProcessedChunk], QualityMetrics]
        """
        print(f"🔄 開始處理文檔: {Path(file_path).name}")
        
        # 1. 提取元資料
        metadata = self.extract_metadata(file_path)
        metadata.processing_status = 'processing'
        
        # 2. 文檔內容提取
        try:
            raw_text = self._extract_text_content(file_path)
            
            # 3. 智能分塊
            chunks = self._semantic_chunking(raw_text, metadata.document_id)
            
            # 4. 品質評估
            quality_metrics = self._assess_quality(raw_text, chunks)
            metadata.quality_score = quality_metrics.overall_score
            metadata.processing_status = 'completed'
            
            print(f"✅ 文檔處理完成 - 品質分數: {quality_metrics.overall_score:.3f}")
            
            return metadata, chunks, quality_metrics
            
        except Exception as e:
            print(f"❌ 文檔處理失敗: {e}")
            metadata.processing_status = 'failed'
            return metadata, [], QualityMetrics(0.0, 0.0, 0.0, 0.0, 0.0)
    
    def _extract_text_content(self, file_path: str) -> str:
        """提取文本內容（兼容多種方法）"""
        file_path = Path(file_path)
        
        # 文本檔案直接讀取
        if file_path.suffix.lower() == '.txt':
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                return f.read()
        
        # 如果 Docling 可用，優先使用
        elif self.doc_converter and file_path.suffix.lower() in ['.pdf', '.docx']:
            try:
                converted_doc = self.doc_converter.convert(str(file_path)).document
                return converted_doc.export_to_text()
            except Exception as e:
                print(f"⚠️  Docling 處理失敗，嘗試替代方法: {e}")
        
        # PDF 替代處理
        if file_path.suffix.lower() == '.pdf':
            try:
                text = ""
                with open(file_path, 'rb') as f:
                    pdf_reader = PyPDF2.PdfReader(f)
                    for page in pdf_reader.pages:
                        text += page.extract_text() + "\n"
                return text
            except Exception as e:
                print(f"⚠️  PDF 替代處理失敗: {e}")
        
        # DOCX 替代處理
        elif file_path.suffix.lower() == '.docx':
            try:
                doc = DocxDocument(file_path)
                text = ""
                for paragraph in doc.paragraphs:
                    text += paragraph.text + "\n"
                return text
            except Exception as e:
                print(f"⚠️  DOCX 替代處理失敗: {e}")
        
        # 如果都失敗，回傳檔案名作為內容
        return f"Unable to extract content from {file_path.name}"

print("✅ EnterpriseDocumentProcessor 定義完成")

✅ EnterpriseDocumentProcessor 定義完成


### 語義分塊演算法實作

In [41]:
# 繼續 EnterpriseDocumentProcessor 類別的方法實作

def _semantic_chunking(self, text: str, document_id: str) -> List[ProcessedChunk]:
    """
    語義分塊演算法
    基於句子相似度的動態分塊策略（兼容版本）
    """
    print("🧠 執行分塊...")
    
    if not text or not text.strip():
        return []
    
    # 如果可用，使用語義分塊
    if self.semantic_model and self.nlp:
        return self._advanced_semantic_chunking(text, document_id)
    else:
        # 降級到基於段落的分塊
        return self._paragraph_based_chunking(text, document_id)

def _advanced_semantic_chunking(self, text: str, document_id: str) -> List[ProcessedChunk]:
    """進階語義分塊（需要 spaCy 和 SentenceTransformers）"""
    # 1. 文本預處理與句子分割
    doc = self.nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 10]
    
    if len(sentences) < 2:
        # 如果句子太少，直接返回單一分塊
        return [ProcessedChunk(
            chunk_id=f"{document_id}_chunk_0",
            document_id=document_id,
            content=text,
            chunk_index=0,
            start_page=1,
            end_page=1,
            word_count=len(text.split()),
            semantic_type='text'
        )]
    
    # 2. 計算句子嵌入向量
    embeddings = self.semantic_model.encode(sentences)
    
    # 3. 計算相鄰句子相似度
    similarities = []
    for i in range(len(embeddings) - 1):
        sim = cosine_similarity([embeddings[i]], [embeddings[i + 1]])[0][0]
        similarities.append(sim)
    
    # 4. 識別語義邊界（相似度局部最小值）
    threshold = self.config['similarity_threshold']
    boundaries = [0]  # 起始邊界
    
    for i in range(1, len(similarities) - 1):
        # 如果當前相似度是局部最小值且低於閾值
        if (similarities[i] < similarities[i-1] and 
            similarities[i] < similarities[i+1] and 
            similarities[i] < threshold):
            boundaries.append(i + 1)
    
    boundaries.append(len(sentences))  # 結束邊界
    
    # 5. 基於邊界生成分塊
    chunks = []
    for i in range(len(boundaries) - 1):
        start_idx = boundaries[i]
        end_idx = boundaries[i + 1]
        
        chunk_sentences = sentences[start_idx:end_idx]
        chunk_text = ' '.join(chunk_sentences)
        word_count = len(chunk_text.split())
        
        # 檢查分塊大小
        if (word_count >= self.config['min_chunk_words'] and 
            word_count <= self.config['max_chunk_words']):
            
            chunk = ProcessedChunk(
                chunk_id=f"{document_id}_chunk_{len(chunks)}",
                document_id=document_id,
                content=chunk_text,
                chunk_index=len(chunks),
                start_page=1,  # 簡化處理，實際應計算頁碼
                end_page=1,
                word_count=word_count,
                semantic_type='text'
            )
            chunks.append(chunk)
    
    print(f"📝 生成 {len(chunks)} 個語義分塊")
    return chunks

def _paragraph_based_chunking(self, text: str, document_id: str) -> List[ProcessedChunk]:
    """基於段落的分塊（降級方案）"""
    print("📝 使用段落分塊（降級方案）")
    
    # 按段落分割
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    
    if not paragraphs:
        # 如果沒有段落，按句子分割
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        paragraphs = sentences
    
    chunks = []
    current_chunk = ""
    current_word_count = 0
    chunk_index = 0
    
    for paragraph in paragraphs:
        para_words = len(paragraph.split())
        
        # 如果當前分塊加上這個段落會超過最大長度
        if current_word_count + para_words > self.config['max_chunk_words'] and current_chunk:
            # 保存當前分塊
            if current_word_count >= self.config['min_chunk_words']:
                chunk = ProcessedChunk(
                    chunk_id=f"{document_id}_chunk_{chunk_index}",
                    document_id=document_id,
                    content=current_chunk.strip(),
                    chunk_index=chunk_index,
                    start_page=1,
                    end_page=1,
                    word_count=current_word_count,
                    semantic_type='text'
                )
                chunks.append(chunk)
                chunk_index += 1
            
            # 開始新分塊
            current_chunk = paragraph
            current_word_count = para_words
        else:
            # 添加到當前分塊
            if current_chunk:
                current_chunk += "\n\n" + paragraph
            else:
                current_chunk = paragraph
            current_word_count += para_words
    
    # 處理最後一個分塊
    if current_chunk and current_word_count >= self.config['min_chunk_words']:
        chunk = ProcessedChunk(
            chunk_id=f"{document_id}_chunk_{chunk_index}",
            document_id=document_id,
            content=current_chunk.strip(),
            chunk_index=chunk_index,
            start_page=1,
            end_page=1,
            word_count=current_word_count,
            semantic_type='text'
        )
        chunks.append(chunk)
    
    print(f"📝 生成 {len(chunks)} 個段落分塊")
    return chunks

def _assess_quality(self, text: str, chunks: List[ProcessedChunk]) -> QualityMetrics:
    """
    多維度文檔品質評估
    基於 ISO 25012 標準
    """
    print("📊 執行品質評估...")
    
    # 1. 準確性評估（基於語言品質）
    accuracy_score = self._calculate_accuracy(text)
    
    # 2. 完整性評估（基於結構完整性）
    completeness_score = self._calculate_completeness(text, chunks)
    
    # 3. 可讀性評估
    readability_score = self._calculate_readability(text)
    
    # 4. 結構評估
    structure_score = self._calculate_structure_score(text)
    
    # 5. 綜合品質分數（加權平均）
    weights = {'accuracy': 0.25, 'completeness': 0.25, 
               'readability': 0.25, 'structure': 0.25}
    
    overall_score = (
        weights['accuracy'] * accuracy_score +
        weights['completeness'] * completeness_score +
        weights['readability'] * readability_score +
        weights['structure'] * structure_score
    )
    
    return QualityMetrics(
        accuracy_score=accuracy_score,
        completeness_score=completeness_score,
        readability_score=readability_score,
        structure_score=structure_score,
        overall_score=overall_score
    )

def _calculate_accuracy(self, text: str) -> float:
    """計算文本準確性（語言品質）"""
    # 簡化實作：基於文本長度和字符分布
    if len(text.strip()) == 0:
        return 0.0
    
    # 基本語言特徵檢查
    if self.nlp:
        # 使用 spaCy 進行精確分析
        doc = self.nlp(text[:1000])  # 取前1000字符進行分析
        
        # 計算有效詞彙比例
        valid_tokens = sum(1 for token in doc if token.is_alpha and not token.is_stop)
        total_tokens = len([token for token in doc if token.is_alpha])
        
        if total_tokens == 0:
            return 0.0
        
        validity_ratio = valid_tokens / total_tokens
        return min(validity_ratio * 1.2, 1.0)  # 稍微提升分數
    else:
        # 降級分析：基於基本統計
        words = text.split()
        if not words:
            return 0.0
        
        # 檢查基本特徵
        alpha_ratio = sum(1 for word in words if word.isalpha()) / len(words)
        avg_word_length = np.mean([len(word) for word in words])
        
        # 簡單評分
        score = alpha_ratio * (min(avg_word_length / 5, 1.0))
        return min(score, 1.0)

def _calculate_completeness(self, text: str, chunks: List[ProcessedChunk]) -> float:
    """計算內容完整性"""
    if not chunks:
        return 0.0
    
    # 基於分塊分布的均勻性
    chunk_sizes = [chunk.word_count for chunk in chunks]
    if not chunk_sizes:
        return 0.0
    
    # 計算分塊大小的變異係數
    mean_size = np.mean(chunk_sizes)
    std_size = np.std(chunk_sizes)
    
    if mean_size == 0:
        return 0.0
    
    cv = std_size / mean_size
    completeness = max(1.0 - cv, 0.0)  # 變異係數越小，完整性越高
    return min(completeness, 1.0)

def _calculate_readability(self, text: str) -> float:
    """計算文本可讀性"""
    try:
        # 使用 Flesch Reading Ease
        flesch_score = textstat.flesch_reading_ease(text)
        # 轉換為 0-1 範圍
        normalized_score = max(0, min(100, flesch_score)) / 100
        return normalized_score
    except:
        # 降級評估
        sentences = text.split('.')
        if not sentences:
            return 0.5
        
        avg_sentence_length = np.mean([len(s.split()) for s in sentences if s.strip()])
        # 理想句子長度為15-25字
        if 15 <= avg_sentence_length <= 25:
            return 0.8
        elif 10 <= avg_sentence_length <= 35:
            return 0.6
        else:
            return 0.4

def _calculate_structure_score(self, text: str) -> float:
    """計算文檔結構分數"""
    # 檢查基本結構元素
    structure_indicators = {
        'has_paragraphs': '\n\n' in text,
        'has_sentences': '.' in text,
        'reasonable_length': 100 < len(text) < 50000,
        'has_capitalization': any(c.isupper() for c in text[:500]),
        'has_sections': any(pattern in text.lower() for pattern in ['abstract', 'introduction', 'conclusion']),
        'has_numbering': bool(re.search(r'^\s*\d+\.', text, re.MULTILINE))
    }
    
    score = sum(structure_indicators.values()) / len(structure_indicators)
    return score

# 將方法添加到 EnterpriseDocumentProcessor 類別
import re

EnterpriseDocumentProcessor._semantic_chunking = _semantic_chunking
EnterpriseDocumentProcessor._advanced_semantic_chunking = _advanced_semantic_chunking
EnterpriseDocumentProcessor._paragraph_based_chunking = _paragraph_based_chunking
EnterpriseDocumentProcessor._assess_quality = _assess_quality
EnterpriseDocumentProcessor._calculate_accuracy = _calculate_accuracy
EnterpriseDocumentProcessor._calculate_completeness = _calculate_completeness
EnterpriseDocumentProcessor._calculate_readability = _calculate_readability
EnterpriseDocumentProcessor._calculate_structure_score = _calculate_structure_score

print("✅ 語義分塊與品質評估方法定義完成（兼容版本）")

✅ 語義分塊與品質評估方法定義完成（兼容版本）


## 🧪 實作練習 1: 單一文檔處理

### 準備測試資料

In [ ]:
# 建立測試用的樣本文檔
sample_text = """
Attention Is All You Need

Abstract
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.

1. Introduction
Recurrent neural networks, long short-term memory and gated recurrent units in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures.

Self-attention, sometimes called intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations.

2. Background
The goal of reducing sequential computation also motivated the creation of the Extended Neural GPU, ByteNet and ConvS2S, all of which use convolutional neural networks as basic building blocks, computing hidden representations in parallel for all input and output positions.
"""

# 建立測試文件
test_dir = PROJECT_ROOT / 'notebooks' / '01_document_ingestion' / 'test_data'
test_dir.mkdir(parents=True, exist_ok=True)

test_file = test_dir / 'transformer_paper_sample.txt'
with open(test_file, 'w', encoding='utf-8') as f:
    f.write(sample_text)

print(f"✅ 測試文件已建立: {test_file}")

### 執行文檔處理

In [43]:
# 🚀 CUDA加速效能測試
import time

print("=== SentenceTransformer CUDA 加速測試 ===")

# 初始化處理器並測試
processor = EnterpriseDocumentProcessor()

if processor.semantic_model:
    # 準備測試文本
    test_texts = [
        "This is a test sentence for performance evaluation.",
        "We are testing the CUDA acceleration capabilities.",
        "Natural language processing with deep learning models.",
        "Transformer architectures have revolutionized NLP.",
        "GPU acceleration provides significant speedup for inference."
    ] * 20  # 100個句子進行測試
    
    print(f"測試文本數量: {len(test_texts)}")
    print(f"使用設備: {processor.semantic_model.device}")
    
    # 執行編碼並測量時間
    start_time = time.time()
    embeddings = processor.semantic_model.encode(test_texts, show_progress_bar=False)
    end_time = time.time()
    
    processing_time = end_time - start_time
    throughput = len(test_texts) / processing_time
    
    print(f"✅ 編碼完成")
    print(f"處理時間: {processing_time:.2f} 秒")
    print(f"處理速度: {throughput:.1f} 句子/秒")
    print(f"向量維度: {embeddings.shape}")
    
    # GPU記憶體使用情況
    if 'cuda' in str(processor.semantic_model.device):
        import torch
        gpu_memory = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU記憶體使用: {gpu_memory:.2f} GB")
        
else:
    print("❌ SentenceTransformer 模型未載入成功")

=== SentenceTransformer CUDA 加速測試 ===
✅ Docling 轉換器初始化完成
🔄 載入語義模型 (優化CUDA設定)...
發現 2 張GPU
使用設備: cuda:1


2025-11-11 15:02:48,301 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅ 語義模型載入完成，設備: cuda:1
✅ spaCy 模型載入完成
✅ DocumentProcessor 初始化完成
測試文本數量: 100
使用設備: cuda:1
✅ 編碼完成
處理時間: 0.06 秒
處理速度: 1783.7 句子/秒
向量維度: (100, 384)
GPU記憶體使用: 0.09 GB


In [44]:
# 初始化處理器
processor = EnterpriseDocumentProcessor()

# 處理測試文檔
metadata, chunks, quality_metrics = processor.process_document(str(test_file))

print("\n📋 文檔元資料:")
print(f"文檔 ID: {metadata.document_id}")
print(f"標題: {metadata.title}")
print(f"文檔類型: {metadata.document_type}")
print(f"檔案大小: {metadata.file_size} bytes")
print(f"處理狀態: {metadata.processing_status}")
print(f"品質分數: {metadata.quality_score:.3f}")

print("\n📝 分塊資訊:")
for i, chunk in enumerate(chunks[:3]):  # 顯示前3個分塊
    print(f"\n分塊 {i+1}:")
    print(f"  ID: {chunk.chunk_id}")
    print(f"  字數: {chunk.word_count}")
    print(f"  內容預覽: {chunk.content[:100]}...")

print(f"\n總共生成 {len(chunks)} 個分塊")

2025-11-11 15:02:51,299 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅ Docling 轉換器初始化完成
🔄 載入語義模型 (優化CUDA設定)...
發現 2 張GPU
使用設備: cuda:1
✅ 語義模型載入完成，設備: cuda:1
✅ spaCy 模型載入完成
✅ DocumentProcessor 初始化完成
🔄 開始處理文檔: transformer_paper_sample.txt
🧠 執行分塊...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📝 生成 2 個語義分塊
📊 執行品質評估...
✅ 文檔處理完成 - 品質分數: 0.690

📋 文檔元資料:
文檔 ID: doc_transformer_paper_sample_1762844574
標題: transformer_paper_sample
文檔類型: paper
檔案大小: 1444 bytes
處理狀態: completed
品質分數: 0.690

📝 分塊資訊:

分塊 1:
  ID: doc_transformer_paper_sample_1762844574_chunk_0
  字數: 106
  內容預覽: Attention Is All You Need Abstract
The dominant sequence transduction models are based on complex re...

分塊 2:
  ID: doc_transformer_paper_sample_1762844574_chunk_1
  字數: 87
  內容預覽: Self-attention, sometimes called intra-attention, is an attention mechanism relating different posit...

總共生成 2 個分塊


### 品質評估結果分析

In [45]:
# 品質指標可視化
quality_data = {
    '維度': ['準確性', '完整性', '可讀性', '結構性', '綜合分數'],
    '分數': [quality_metrics.accuracy_score, 
            quality_metrics.completeness_score,
            quality_metrics.readability_score, 
            quality_metrics.structure_score,
            quality_metrics.overall_score],
    '閾值': [0.7, 0.8, 0.6, 0.7, 0.7]
}

fig = go.Figure()

# 實際分數
fig.add_trace(go.Bar(
    name='實際分數',
    x=quality_data['維度'],
    y=quality_data['分數'],
    marker_color='lightblue'
))

# 閾值線
fig.add_trace(go.Scatter(
    name='品質閾值',
    x=quality_data['維度'],
    y=quality_data['閾值'],
    mode='markers+lines',
    line=dict(color='red', dash='dash'),
    marker=dict(size=8, color='red')
))

fig.update_layout(
    title='文檔品質評估結果',
    xaxis_title='品質維度',
    yaxis_title='分數 [0-1]',
    yaxis=dict(range=[0, 1]),
    showlegend=True
)

fig.show()

print("\n📊 品質評估詳細結果:")
for dim, score, threshold in zip(quality_data['維度'], quality_data['分數'], quality_data['閾值']):
    status = "✅ 通過" if score >= threshold else "❌ 未通過"
    print(f"{dim}: {score:.3f} (閾值: {threshold}) {status}")


📊 品質評估詳細結果:
準確性: 0.757 (閾值: 0.7) ✅ 通過
完整性: 0.902 (閾值: 0.8) ✅ 通過
可讀性: 0.103 (閾值: 0.6) ❌ 未通過
結構性: 1.000 (閾值: 0.7) ✅ 通過
綜合分數: 0.690 (閾值: 0.7) ❌ 未通過


## 🧪 實作練習 2: 批次文檔處理

### 批次處理管線

In [46]:
class BatchDocumentProcessor:
    """
    批次文檔處理器
    支援目錄級別的文檔處理
    """
    
    def __init__(self, processor: EnterpriseDocumentProcessor):
        self.processor = processor
        self.processing_results = []
    
    def process_directory(self, directory_path: str, file_patterns: List[str] = ['*.pdf', '*.txt', '*.docx']) -> Dict:
        """
        處理目錄中的所有符合條件的文檔
        """
        directory = Path(directory_path)
        if not directory.exists():
            raise ValueError(f"目錄不存在: {directory_path}")
        
        # 收集需要處理的文件
        files_to_process = []
        for pattern in file_patterns:
            files_to_process.extend(directory.glob(pattern))
        
        print(f"🔍 找到 {len(files_to_process)} 個文件待處理")
        
        # 批次處理
        results = {
            'processed_files': [],
            'failed_files': [],
            'total_chunks': 0,
            'average_quality': 0.0
        }
        
        quality_scores = []
        
        for i, file_path in enumerate(files_to_process):
            print(f"\n📄 處理 {i+1}/{len(files_to_process)}: {file_path.name}")
            
            try:
                metadata, chunks, quality = self.processor.process_document(str(file_path))
                
                if metadata.processing_status == 'completed':
                    results['processed_files'].append({
                        'file_path': str(file_path),
                        'metadata': metadata.to_dict(),
                        'chunk_count': len(chunks),
                        'quality_score': quality.overall_score
                    })
                    results['total_chunks'] += len(chunks)
                    quality_scores.append(quality.overall_score)
                else:
                    results['failed_files'].append(str(file_path))
                    
            except Exception as e:
                print(f"❌ 處理失敗: {e}")
                results['failed_files'].append(str(file_path))
        
        # 計算平均品質
        if quality_scores:
            results['average_quality'] = np.mean(quality_scores)
        
        self.processing_results = results
        return results
    
    def generate_report(self) -> pd.DataFrame:
        """
        生成處理報告
        """
        if not self.processing_results:
            return pd.DataFrame()
        
        report_data = []
        for file_info in self.processing_results['processed_files']:
            metadata = file_info['metadata']
            report_data.append({
                '文件名': Path(file_info['file_path']).name,
                '文檔類型': metadata['document_type'],
                '檔案大小(KB)': round(metadata['file_size'] / 1024, 2),
                '分塊數量': file_info['chunk_count'],
                '品質分數': round(file_info['quality_score'], 3),
                '處理狀態': metadata['processing_status']
            })
        
        return pd.DataFrame(report_data)

print("✅ BatchDocumentProcessor 定義完成")

✅ BatchDocumentProcessor 定義完成


### 執行批次處理

In [47]:
# 建立更多測試文件
test_docs = [
    ("bert_paper_sample.txt", """
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding

Abstract
We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers.

1. Introduction
Language model pre-training has been shown to be effective for improving many natural language processing tasks. These include sentence classification tasks such as natural language inference and paraphrasing, which aim to predict the relationships between sentences by analyzing them holistically.
"""),
    ("gpt3_paper_sample.txt", """
Language Models are Few-Shot Learners

Abstract
Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples.

By contrast, humans can generally perform a new language task from only a few examples or from simple instructions. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance.
""")
]

# 建立測試文件
for filename, content in test_docs:
    test_file = test_dir / filename
    with open(test_file, 'w', encoding='utf-8') as f:
        f.write(content)

print(f"✅ 建立了 {len(test_docs)} 個額外測試文件")

# 執行批次處理
batch_processor = BatchDocumentProcessor(processor)
batch_results = batch_processor.process_directory(str(test_dir))

print("\n📊 批次處理結果:")
print(f"成功處理: {len(batch_results['processed_files'])} 個文件")
print(f"處理失敗: {len(batch_results['failed_files'])} 個文件")
print(f"總分塊數: {batch_results['total_chunks']}")
print(f"平均品質: {batch_results['average_quality']:.3f}")

✅ 建立了 2 個額外測試文件
🔍 找到 3 個文件待處理

📄 處理 1/3: transformer_paper_sample.txt
🔄 開始處理文檔: transformer_paper_sample.txt
🧠 執行分塊...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📝 生成 2 個語義分塊
📊 執行品質評估...
✅ 文檔處理完成 - 品質分數: 0.690

📄 處理 2/3: bert_paper_sample.txt
🔄 開始處理文檔: bert_paper_sample.txt
🧠 執行分塊...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📝 生成 1 個語義分塊
📊 執行品質評估...
✅ 文檔處理完成 - 品質分數: 0.731

📄 處理 3/3: gpt3_paper_sample.txt
🔄 開始處理文檔: gpt3_paper_sample.txt
🧠 執行分塊...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📝 生成 1 個語義分塊
📊 執行品質評估...
✅ 文檔處理完成 - 品質分數: 0.735

📊 批次處理結果:
成功處理: 3 個文件
處理失敗: 0 個文件
總分塊數: 4
平均品質: 0.719


### 批次處理報告

In [48]:
# 生成處理報告
report_df = batch_processor.generate_report()

print("📋 批次處理報告:")
print(report_df.to_string(index=False))

# 品質分佈可視化
if not report_df.empty:
    fig = px.bar(
        report_df, 
        x='文件名', 
        y='品質分數',
        title='各文檔品質分數比較',
        color='品質分數',
        color_continuous_scale='Viridis'
    )
    
    fig.add_hline(y=0.7, line_dash="dash", line_color="red", 
                  annotation_text="品質閾值 (0.7)")
    
    fig.update_layout(
        xaxis_tickangle=-45,
        height=500
    )
    
    fig.show()

    # 統計摘要
    print("\n📈 統計摘要:")
    print(f"平均品質分數: {report_df['品質分數'].mean():.3f}")
    print(f"品質分數標準差: {report_df['品質分數'].std():.3f}")
    print(f"通過品質閾值(0.7)的文檔: {(report_df['品質分數'] >= 0.7).sum()}/{len(report_df)}")

📋 批次處理報告:
                         文件名  文檔類型  檔案大小(KB)  分塊數量  品質分數      處理狀態
transformer_paper_sample.txt paper      1.41     2 0.690 completed
       bert_paper_sample.txt paper      0.73     1 0.731 completed
       gpt3_paper_sample.txt paper      0.57     1 0.735 completed



📈 統計摘要:
平均品質分數: 0.719
品質分數標準差: 0.025
通過品質閾值(0.7)的文檔: 2/3


## 🎓 學習檢核與下一步

### 學習成果檢核

完成本模組後，您已經掌握了：

✅ **文檔攝取架構設計**
- 標準化元資料模型
- 可擴展的處理器架構
- 錯誤處理機制

✅ **Docling 整合應用**
- 多格式文檔解析
- 內容提取與結構化
- 元資料自動提取

✅ **語義分塊演算法**
- 基於相似度的邊界檢測
- 動態分塊策略
- 語義連貫性保持

✅ **品質評估機制**
- 多維度品質指標
- 自動化評分算法
- 閾值檢查與報告

### 進階挑戰練習

1. **多語言支援**: 擴展處理器支援中文文檔
2. **表格處理**: 加強表格內容的結構化提取
3. **圖像處理**: 整合 OCR 處理掃描文檔
4. **並發處理**: 實作 async 並發處理能力

### 下一模組預告

**模組 2: 元資料管理與索引建立**
- 企業級元資料 schema 設計
- 向量索引建立與管理
- 混合檢索策略實作

In [49]:
# 儲存本次練習結果
results_dir = PROJECT_ROOT / 'notebooks' / '01_document_ingestion' / 'exercises'
results_dir.mkdir(parents=True, exist_ok=True)

# 儲存處理結果
import pickle

with open(results_dir / 'exercise_results.pkl', 'wb') as f:
    pickle.dump({
        'metadata': metadata,
        'chunks': chunks,
        'quality_metrics': quality_metrics,
        'batch_results': batch_results
    }, f)

# 儲存報告
report_df.to_csv(results_dir / 'processing_report.csv', index=False)

print("✅ 練習結果已儲存")
print(f"📂 結果位置: {results_dir}")
print("\n🎉 模組 1 完成！準備進入模組 2: 元資料管理與索引建立")

✅ 練習結果已儲存
📂 結果位置: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance/notebooks/01_document_ingestion/exercises

🎉 模組 1 完成！準備進入模組 2: 元資料管理與索引建立


In [50]:
# 儲存本次練習結果
results_dir = PROJECT_ROOT / 'notebooks' / '01_document_ingestion' / 'exercises'
results_dir.mkdir(parents=True, exist_ok=True)

# 儲存處理結果
import pickle

with open(results_dir / 'exercise_results.pkl', 'wb') as f:
    pickle.dump({
        'metadata': metadata,
        'chunks': chunks,
        'quality_metrics': quality_metrics,
        'batch_results': batch_results
    }, f)

# 儲存報告
report_df.to_csv(results_dir / 'processing_report.csv', index=False)

print("✅ 練習結果已儲存")
print(f"📂 結果位置: {results_dir}")
print("\n🎉 模組 1 完成！準備進入模組 2: 元資料管理與索引建立")

✅ 練習結果已儲存
📂 結果位置: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance/notebooks/01_document_ingestion/exercises

🎉 模組 1 完成！準備進入模組 2: 元資料管理與索引建立
